# Expression remapper

This notebook is a simple app that allows users to map different facial expressions and motions to keyboard inputs. I aim to use this to play a video game like Dark Souls as a test.

Using MediaPipe, and following their [face landmark detection guide](https://developers.google.com/mediapipe/solutions/vision/face_landmarker/python).

## Install dependencies

#### Mediapipe

In [ ]:
!pip install mediapipe
# opencv as well

## Import dependencies

In [1]:
# Mediapipe for face landmark detection
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# OpenCV for drawing utilities and webcam input
import cv2

## Load the model

In [2]:
model_path = 'models/face_landmarker.task'

## Utility functions for extracting head orientation

In [21]:
import math

# Real yaw pitch and roll
yaw = 0.0 # Rotate left/right
pitch = 0.0 # Tilt up/down
roll = 0.0 # Tilt left/right

# Calibrated yaw pitch and roll values
calib_yaw = 0
calib_pitch = 0
calib_roll = 0

# The rest positions, used to calculate the calibrated values
rest_yaw = 0.0
rest_pitch = 0.0
rest_roll = 0.0

# Extract the yaw, pitch, and roll of the head from the facial transformation matrix
# (yaw, pitch, and roll are easier for me to work with than a matrix)
def update_yaw_pitch_roll(mtrix):
    # Get access to the global variables
    global yaw
    global pitch
    global roll

    # Calculate the yaw pitch and roll from the transformation matrix (conversion)
    if mtrix[0][0] == 1.0:
        yaw = math.atan2(mtrix[0][2], mtrix[2][3])
        pitch = 0
        roll = 0

    elif mtrix[0][0] == -1.0:
        yaw = math.atan2(mtrix[0][0], mtrix[2][3])
        pitch = 0
        roll = 0

    else:
        yaw = math.atan2(-mtrix[2][0], mtrix[0][0])
        pitch = math.atan2(-mtrix[1][2], mtrix[1][1])
        roll = math.asin(mtrix[1][0])

    # Calculate the CALIBRATED yaw pitch roll values
    global calib_yaw
    global calib_pitch
    global calib_roll
    calib_yaw = yaw - rest_yaw
    calib_pitch = pitch - rest_pitch
    calib_roll = roll - rest_roll
    
    # Yaw (look left/right)
    print("yaw is: {}".format(math.degrees(calib_yaw)))
    if calib_yaw > 0:
        print("looking left")
    elif calib_yaw < 0:
        print("looking right")

#         # Pitch (look up/down)
#         print("pitch is: {}".format(math.degrees(calib_pitch)))
#         if calib_pitch > 0:
#             print("looking down")
#         elif calib_pitch < 0:
#             print("looking up")

#         # Roll (rilt left/right)
#         print("roll is: {}".format(math.degrees(calib_roll)))
#         if calib_roll > 0:
#             print("looking rolled right")
#         elif calib_roll < 0:
#             print("looking rolled left")
    
def calibrate_yaw_pitch_roll():
    # Get access to global rest pos variables (I don't like python..)
    # then set them to the current yaw, pitch, and roll of the user's head.
    global yaw
    global pitch
    global roll
    global rest_yaw
    global rest_pitch
    global rest_roll
    rest_yaw = yaw
    rest_pitch = pitch
    rest_roll = roll
    
    # Print results
    print("resting yaw is: " + str(rest_yaw))
    print("resting pitch is: " + str(rest_pitch))
    print("resting roll is: " + str(rest_roll))
    

## Main program

In [22]:
import mediapipe as mp

BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
FaceLandmarkerResult = mp.tasks.vision.FaceLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode
    

# Create a face landmarker instance with the live stream mode:
def print_result(result: FaceLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    
    # If any landmarks have been detected..
    if result.face_landmarks:
        
        # Get the matrix from face landmarker
        transf_matrix = result.facial_transformation_matrixes[0]
        update_yaw_pitch_roll(transf_matrix)   
    

options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result,
    output_facial_transformation_matrixes=True, ## TRY CHANGING THIS
    output_face_blendshapes=False) ## TRY CHANGING THIS

with FaceLandmarker.create_from_options(options) as landmarker:
    # The landmarker is initialized. Use it here.
    # Use OpenCV’s VideoCapture to start capturing from the webcam.
    video = cv2.VideoCapture(0)
    
    # Initialise the frame timestamp (MAKE THIS IN ms?)
    frame_timestamp = 0
    
    # Tell the user to calibrate
    print("---- Calibration -----")
    print("Please look straight ahead at your screen, then press ENTER when comfortable")
    
    # Create a loop to read the latest frame from the camera using VideoCapture#read()
    while video.isOpened():
        ret, frame = video.read()
        
        # Convert the frame received from OpenCV to a MediaPipe Image object.
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        
        # Send live image data to perform face landmarking.
        # The results are accessible via the `result_callback` provided in
        # the `PoseLandmarkerOptions` object.
        # The pose landmarker must be created with the live stream mode.
        landmarker.detect_async(mp_image, frame_timestamp)
        
        # Increment the timestamp
        frame_timestamp += 1
        
        # Draw the output
        cv2.imshow('window', frame)
        
        # Listen for keyboard input
        keyPressed = cv2.waitKey(5)
        if keyPressed == ord('q'): # 'q': Break out of loop and exit program
            break
        elif keyPressed == 13: # 'Enter': Calibrate yaw+pitch+roll
            calibrate_yaw_pitch_roll()

# Clean up
video.release()
cv2.destroyAllWindows()

---- Calibration -----
Please look straight ahead at your screen, then press ENTER when comfortable
yaw is: 4.8039768463402
looking left
yaw is: 2.819565313959531
looking left
yaw is: 1.4881417892209963
looking left
yaw is: 0.323732610899192
looking left
yaw is: -0.8624634229820667
looking right
yaw is: -1.698729097495108
looking right
yaw is: -2.6209745462333083
looking right
yaw is: -3.3763666374013437
looking right
yaw is: -5.037169558966096
looking right
yaw is: -6.788392845699759
looking right
yaw is: -7.947004205989876
looking right
yaw is: -8.898767264602634
looking right
yaw is: -9.844536134370548
looking right
yaw is: -10.245804621749643
looking right
yaw is: -10.804597914523908
looking right
yaw is: -11.00126754856813
looking right
yaw is: -11.032177890058412
looking right
yaw is: -11.050573268083413
looking right
yaw is: -10.978715558387687
looking right
yaw is: -10.60341676808547
looking right
yaw is: -10.19228887519693
looking right
yaw is: -9.4854359153975
looking right
y

yaw is: -7.263974812810472
looking right
yaw is: -8.989513985910339
looking right
yaw is: -11.136739713070977
looking right
yaw is: -12.971905541445134
looking right
yaw is: -14.28107813302106
looking right
yaw is: -15.471035889901097
looking right
yaw is: -16.64667087437354
looking right
yaw is: -17.458572244290906
looking right
yaw is: -18.05063308086417
looking right
yaw is: -18.491545855890003
looking right
yaw is: -18.873805271996346
looking right
yaw is: -19.139551782866967
looking right
yaw is: -19.39273372672227
looking right
resting yaw is: -0.12075140032124887
resting pitch is: -0.034318608792123166
resting roll is: 0.056009253820730666
yaw is: -0.01962238049766479
looking right
yaw is: -0.06641881782831272
looking right
yaw is: 0.011146119374398343
looking left
yaw is: 0.12110498721927296
looking left
yaw is: 0.24934599032855684
looking left
yaw is: 0.3360787910144995
looking left
yaw is: 0.39989154558223805
looking left
yaw is: 0.49228237187544954
looking left
yaw is: 0.542

In [13]:
-34 + (-10 * -1)

-24

In [14]:
-34 - -10

-24

In [15]:
-34 - 10

-44

In [16]:
-34 + (10 * -1)

-44